# CIFAR10: Training a classifer

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lento234/ml-tutorials/blob/main/01-basics/CIFAR10.ipynb)

**References**:
- https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

**Runtime setup: GPU accelerator at Google colab:**

1. On the main menu, click **Runtime** and select **Change runtime type**. 
2. Select **GPU** as the hardware accelerator.


![steps](../images/steps.png)

In [ ]:
!nvidia-smi

**Table of content**

1. [Setup environment](#setup)
2. [Load and pre-process the dataset](#load)
3. [Define the CNN model](#define)
4. [Define the loss function and optimizer](#loss)
5. [Train the model on **training** dataset](#train)
6. [Test/Validate the model on **test** dataset](#validate)

**CIFAR10 Dataset**

The dataset consists of `3x32x32` images of 10 difference classes:

    airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck.

![cifar10](../images/cifar10.png)

<a id='setup'></a>
## 1. Setup environment

### Load packages / modules

In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets

In [ ]:
mpl.style.use('seaborn-poster')
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['figure.figsize'] = 5 * np.array([1.618033988749895, 1])

In [ ]:
# Reproducability
seed = 234
np.random.seed(seed)
torch.random.manual_seed(seed);

### Setup computing platform: GPU accelerator

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
assert(device=="cuda"), "GPU not available, try again !!!!"
print(f"Computing with {device}")

### Define the hyper-parameters

In [ ]:
batch_size = 16
num_workers = 4
num_epochs = 5
learning_rate = 0.001
momentum = 0.9

<a id='load'></a>
## 2. Load and pre-process data

### Define preprocessing algorithm

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(), # convert data to pytorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalize dataset for each channel
])

### Load training and test dataset

In [ ]:
# Download train and test dataset
train_dataset = datasets.CIFAR10(root='./data', train=True,
                                 download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, 
                                 download=True, transform=transform)

# Dataset sampler (shuffle, distributed loading)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
                                          shuffle=False, num_workers=num_workers)

classes = np.array(['plane', 'car', 'bird', 'cat', 'deer',
                    'dog', 'frog', 'horse', 'ship', 'truck'])

print(f"Number of examples: train = {len(train_dataset)}, test = {len(test_dataset)}")

In [ ]:
def imshow(images, labels):    
    plt.figure(figsize=(8, 8))
    for i in range(16):
        plt.subplot(4,4,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        img = images[i] / 2 + 0.5 # unnormalize
        plt.imshow(np.transpose(img.numpy(), (1, 2, 0)), cmap=plt.cm.binary)
        plt.title(labels[i], fontsize=14)
    
# get some random training images
images, labels = next(iter(train_loader))

# show images
imshow(images, classes[labels])

<a id=define></a>
## 3. Define the CNN model

![network_architecture](../images/network_architecture.png)

**Architecture:**

- Input: An image of `n_channels=3`.
- Two layer stacks of 2D convolutional layers (`Conv2d` with `kernel_size=5`) with rectified linear activation (`ReLU`) followed by a  2D max pooling (`MaxPool2D` with `kernel_size=2` and `stride=2`)
- Three layer stacks of Fully-connected layers (`Linear`) with ReLU activaton.
- Output: 10-dimensional vector defining the activation of each class

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define network
        self.layer1 = nn.Sequential(nn.Conv2d(3, 6, kernel_size=5),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 =  nn.Sequential(nn.Conv2d(6, 16, kernel_size=5),
                                     nn.ReLU(),
                                     nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(nn.Flatten(),
                                    nn.Linear(16 * 5 * 5, 120),
                                    nn.ReLU())
        self.layer4 = nn.Sequential(nn.Linear(120, 84),
                                    nn.ReLU())
        self.layer5 = nn.Linear(84, 10)
        

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return x

In [ ]:
model = Net() # construct
model = model.to(device) # move model to device (GPU)
print(model)

<a id=loss></a>
## 4. Define the loss function and optimizer

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

print("loss:", criterion)
print("optimizer:", optimizer)

<a id=train></a>
## 5. Train the model on **training** dataset

In [ ]:
loss_history = []
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (x_train, y_train) in tqdm(enumerate(train_loader), 
                                              desc=f"[Epoch {epoch}]",
                                              total=len(train_loader)):
        # send batch to GPU
        x_train, y_train = x_train.to(device), y_train.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward-pass
        y_pred = model(x_train)
        
        # backward propogation
        loss = criterion(y_pred, y_train)
        loss.backward()
        
        # update weights
        optimizer.step()

        # log stats
        running_loss += loss.item()
    loss_history.append(running_loss / len(train_loader))
print('Finished Training')

In [ ]:
fig, ax = plt.subplots()
ax.plot(loss_history, '.-')
ax.set(xlabel='epoch', ylabel='loss',
       title='loss history');

<a id="validate"></a>
## 5. Test/Validate the model on **test** dataset

### 5.1 Predict using the trained model

In [ ]:
images, labels = next(iter(test_loader))

# predict
prediction = model(images.to(device))
prediction = prediction.cpu() # gpu -> cpu
predicted_labels = torch.argmax(prediction, 1).detach()

# print images
fig = imshow(images, [r"{} {}".format(classes[pred], "$✓$" if pred == gt else r"$\times$")
                for pred, gt in zip(predicted_labels, labels)])

### 5.2. Accuracy of the model on **test** dataset

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, total=len(test_loader)):
        prediction = model(images.to(device)).cpu()
        predicted_labels = torch.argmax(prediction, 1)
        total += predicted_labels.size(0)
        correct += (predicted_labels == labels).sum().item()

print('\nAccuracy on {} test images: {}%'.format(
    len(test_loader)*batch_size,
     100 * correct / total))

### 5.3 Accuracy of the model per class

In [ ]:
class_correct = np.zeros(len(classes))
class_total = np.zeros(len(classes))
with torch.no_grad():
    for images, labels in tqdm(test_loader, total=len(test_loader)):
        prediction = model(images.to(device)).cpu()
        predicted_labels = torch.argmax(prediction, 1)
        c = (predicted_labels == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print("\nAccuracy\n-------------------------")
sorted_idx = np.argsort(class_correct/class_total)[::-1]
for i in sorted_idx:
    print('Accuracy of {:5s}: {}%'.format(
           classes[i], 100 * class_correct[i] / class_total[i]))